In [65]:
import numpy as np
import pandas as pd

In [49]:
!pip install tensorflow

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding #, CuDNNGRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [67]:
import tensorflow as tf

In [68]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
#geçerli gpu var mı

Num GPUs Available:  0


In [3]:
dataset = pd.read_csv('hepsiburada.csv')

In [4]:
dataset

,Rating,Review
0,1,3 yıldır tık demedi. :)
1,1,3 yıldır kullanıyorum müthiş
2,1,Ürün bugün elime geçti çok fazla inceleme fırs...
3,1,Almaya karar verdim. Hemencecik geldi. Keyifle...
4,1,Günlük kullanımınızı çok çok iyi karsılıyor kı...
...,...,...
243492,1,fiyatına göre güzel
243493,1,Ürün kullanışlı iş görüyor fazlasıyla eşime al...
243494,1,"Hızlı Kargo, güzel ürün"
243495,1,telefon başarılı hızlı bir cihaz sadece beyaz...


In [5]:
dataset['Rating'].values

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [6]:
dataset['Rating'].values.tolist()

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [7]:
target = dataset['Rating'].values.tolist() #verileri panda data nesnesinden listeye çeviriyoruz
data = dataset['Review'].values.tolist()

In [8]:
cutoff = int(len(data) * 0.80) #verilerin %80'nini train kalanı test olarak ayırdık
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target[:cutoff], target[cutoff:]

In [9]:
x_train[500]

'ürün sipariş verdim 2 gün içinde elime ulaştı her zaman ki gibi kullanışlı bi ürün daha once de bu mouse dan almıştım.bu yüzden tereddütsüz aldım . alacak olanlara öneririm'

In [10]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [11]:
y_train[800]

1

In [12]:
num_words = 10000 #datasetteki en sık kullanılan 10bin kelimenin dictinary nesnesini oluşturduk
tokenizer = Tokenizer(num_words=num_words)

In [13]:
tokenizer.fit_on_texts(data)

In [14]:
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(x_train) #kelimeleri sayısal değerlere çevidik

In [16]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [17]:
print(x_train_tokens[800])

[19, 341, 52, 194, 30, 54, 7992, 55, 209, 603, 7887, 3, 36, 114, 164, 479, 85, 1, 1682, 782, 30, 2, 380, 326, 3, 1, 81, 132, 562, 9, 10]


In [18]:
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [19]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [20]:
np.mean(num_tokens) #her yorumun ortalama kaç kelimeden oluştuğuna bakıyoruz

20.744703220162876

In [21]:
np.max(num_tokens) #her yorumun maksimum kaç kelimeden oluştuğuna bakıyoruz

295

In [22]:
np.argmax(num_tokens) #en uzun yorumun indisi

21941

In [23]:
x_train[21941] #en uzun yorum

'Özellikle bu kısma yazıyorum iyice okuyunuz,cihazın hızı çok iyi.Isınma normal boyutlarda.Kamerası elinizi sabit tutarsanız ve gündüz çok net gece ise çok net çekmez.Görüntülü konuşma özelliği yok ancak uygulama ile olur,anten çekimi gayet iyi.Parmak izi okuyucusu gayet hızlı bazen tuşu silmenizi istiyor ve okuyamıyor kirden vs...Ön kamera da tatmin edici,çekim esnasında ekran beyaz ışık vererek flaş görevi görür.Batarya bana 1 hafta gidiyor sık kullanımda ise 2-3 gün gidiyor.Hızlı şarj 1 saatte doluyor , şekilleri ele oturuyor ve şık bir görüntü var.Telefonu aldığım gün gittim ve ilk girdiğim yerden ekran koruyucu ve kılıf buldum.Kulaklık sesi çok net ve yüksek ancak kendi hoparlörü biraz zayıf sesi.Ekrana bakarken açık kalma özelliği yok.Diğer akıllı cihaz özellikleri %90 ı bu cihazda mevcut.Güç tasarrufu 2 ayrı modu var ve çok başarılı çalışıyor.4gb ram var genelde yarısı boş kalıyor.Bir de yeni cihazların çoğu titreşimi az ve sesi de az çıkıyor.Bu cihaz da içine dahil...Bu cihazla

In [24]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) #ortalamadan 2 standart sapma sonrasına kadar olan sayıyı buluyoruz
max_tokens = int(max_tokens)
max_tokens

59

In [25]:
np.sum(num_tokens < max_tokens) / len(num_tokens) #kelimelerimizin yüzde kaçı bu aralıkta diye bakıyoruz

0.9597982726686571

In [26]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens) #yorumları max_tokens uzunluğuna pad ediyoruz. Yani daha kısa olanlara sıfır ekliyoruz daha uzun olanları kırpıyoruz

In [27]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [28]:
x_train_pad.shape

(194797, 59)

In [29]:
x_test_pad.shape

(48700, 59)

In [30]:
np.array(x_train_tokens[800])

array([  19,  341,   52,  194,   30,   54, 7992,   55,  209,  603, 7887,
          3,   36,  114,  164,  479,   85,    1, 1682,  782,   30,    2,
        380,  326,    3,    1,   81,  132,  562,    9,   10])

In [31]:
x_train_pad[800]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,   19,  341,   52,  194,   30,
         54, 7992,   55,  209,  603, 7887,    3,   36,  114,  164,  479,
         85,    1, 1682,  782,   30,    2,  380,  326,    3,    1,   81,
        132,  562,    9,   10])

In [32]:
# sayısallaştırdığımız yorumlardan tekrar metni verecek fonksiyonu oluşturuyoruz
idx = tokenizer.word_index 
inverse_map = dict(zip(idx.values(), idx.keys()))

In [33]:
def tokens_to_string(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [34]:
x_train[800]

'ürünü alalı 3 hafta kadar oldu. aralıksız kullanıyorum bilgisyarım sürekli açık durur ve ben günde yaklaşık 12 saat başındayım mousesu çok kullanırım. şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim.'

In [35]:
tokens_to_string(x_train_tokens[800])
#görüldüğü gibi sık kullanılmayan kelimeler çıkartılarak tokenlaştırma yapılmış

'ürünü alalı 3 hafta kadar oldu aralıksız kullanıyorum sürekli açık durur ve ben günde yaklaşık 12 saat çok kullanırım şimdiye kadar bir problem yaşamadım ve çok memnunum almak isteyenlere tavsiye ederim'

In [36]:
model = Sequential() #keras sequantial nesnesini kelime vektörlerini elde etmek için oluşturuyoruz

In [37]:
embedding_size = 50 
#vektör boyutu. (Burada word2vec veya glove vektörleri değil bunlar yerine kendi vektörleştirmemizi kullandık)

In [38]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='embedding_layer'))

In [40]:
#gpu desteği varsa bu satırları kullan
'''
model.add(CuDNNGRU(units=16, return_sequences=True)) 
model.add(CuDNNGRU(units=8, return_sequences=True)) 
model.add(CuDNNGRU(units=4)) 
model.add(Dense(1, activation='sigmoid')) 
'''

"\nmodel.add(CuDNNGRU(units=16, return_sequences=True)) \nmodel.add(CuDNNGRU(units=8, return_sequences=True)) \nmodel.add(CuDNNGRU(units=4)) \nmodel.add(Dense(1, activation='sigmoid')) \n"

In [41]:
#gpu desteği yoksa bu satırları kullan (çok daha yavaş işlem yapar)
model.add(GRU(units=16, return_sequences=True)) #burada return_sequences=true denmesinin nedeni ard arda katman oluşturulmasıdır.
model.add(GRU(units=8, return_sequences=True)) #eğer cpu varsa ve tensorflow gpu için kurulu ise GRU yerine CuDNNGRU kullanılması hızı 10 kat arttırır.
model.add(GRU(units=4)) #son katmanda bir sonraki tekbir nöron olacağından return_sequences=False yapıldı
model.add(Dense(1, activation='sigmoid')) #Dense çıkış katmanındaki tek bir nörondan oluşan kısım.

In [42]:
#from keras.optimizers import adam_v2

In [43]:
optimizer = Adam(lr=1e-3)
#optimizer = adam_v2.Adam(learning_rate=1e-3)

C:\Users\ysfbil\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
?model.compile

In [44]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_layer (Embedding)  (None, 59, 50)           500000    
                                                                 
 gru (GRU)                   (None, 59, 16)            3264      
                                                                 
 gru_1 (GRU)                 (None, 59, 8)             624       
                                                                 
 gru_2 (GRU)                 (None, 4)                 168       
                                                                 
 dense (Dense)               (None, 1)                 5         
                                                                 
Total params: 504,061
Trainable params: 504,061
Non-trainable params: 0
_________________________________________________________________


In [49]:
#kütüpanenin yeni sürümünde bunlar np.array olmalı
x_train=np.array(x_train)
x_test=np.array(x_test)
y_train=np.array(y_train)
y_test=np.array(y_test)

In [47]:
model.fit(np.array(x_train_pad), np.array(y_train), epochs=5, batch_size=256) #5 defa hesaplıyor. Veriyi 256'lık parçalara böl

Epoch 1/5
761/761 [==============================] - 80s 97ms/step - loss: 0.2059 - accuracy: 0.9490
Epoch 2/5
761/761 [==============================] - 76s 100ms/step - loss: 0.0980 - accuracy: 0.9691
Epoch 3/5
761/761 [==============================] - 76s 100ms/step - loss: 0.0726 - accuracy: 0.9776
Epoch 4/5
761/761 [==============================] - 77s 102ms/step - loss: 0.0570 - accuracy: 0.9832
Epoch 5/5
761/761 [==============================] - 75s 98ms/step - loss: 0.0444 - accuracy: 0.9876


In [50]:
result = model.evaluate(x_test_pad, y_test)

1522/1522 [==============================] - 22s 14ms/step - loss: 0.1538 - accuracy: 0.9515


In [51]:
result[1]

0.9515400528907776

In [52]:
y_pred = model.predict(x=x_test_pad[0:1000]) #ilk 1000 test verisini tahmin ediyoruz
y_pred = y_pred.T[0] #sütun matrisi satır matrise çevirdik

32/32 [==============================] - 2s 14ms/step


In [53]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred]) #0.5'ten büyük olanları 1 diğerini 0 yapıyoruz

In [54]:
cls_true = np.array(y_test[0:1000]) #ilk 1000 veri etiketini alıyoruz

In [55]:
incorrect = np.where(cls_pred != cls_true) #farklı olanları tespit ediyoruz
incorrect = incorrect[0] #tupple'ı listeye çevirdik

In [56]:
len(incorrect) #hatalı tahmin sayısı

65

In [57]:
idx = incorrect[0]
idx #ilk hatalı tahmin

5

In [58]:
text = x_test[idx]
text #hatalı tahmin edilen yorum

'Ürün güzel ama teslimat çok geç yapıldı.'

In [59]:
y_pred[idx] #tahmin oranı

0.9407386

In [60]:
cls_true[idx] #gerçekte işaretlenen

0

In [61]:
#tahmin edilecek verilerimiz
text1 = "bu ürün çok iyi herkese tavsiye ederim"
text2 = "kargo çok hızlı aynı gün elime geçti"
text3 = "büyük bir hayal kırıklığı yaşadım bu ürün bu markaya yakışmamış"
text4 = "mükemmel"
text5 = "tasarımı harika ancak kargo çok geç geldi ve ürün açılmıştı tavsiye etmem"
text6 = "hiç resimde gösterildiği gibi değil"
text7 = "kötü yorumlar gözümü korkutmuştu ancak hiçbir sorun yaşamadım teşekkürler"
text8 = "hiç bu kadar kötü bir satıcıya denk gelmemiştim ürünü geri iade ediyorum"
text9 = "tam bir fiyat performans ürünü"
text10 = "beklediğim gibi çıkmadı"
texts = [text1, text2, text3, text4, text5, text6, text7, text8, text9, text10]

In [62]:
tokens = tokenizer.texts_to_sequences(texts) #sayısallaştırdık

In [63]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens) #padding yapıyoruz
tokens_pad.shape

(10, 59)

In [64]:
model.predict(tokens_pad) #tahminlerimiz

1/1 [==============================] - 0s 50ms/step


array([[0.9976868 ],
       [0.9945425 ],
       [0.11451596],
       [0.9966487 ],
       [0.02800052],
       [0.09372111],
       [0.9962551 ],
       [0.01520283],
       [0.99667364],
       [0.30481642]], dtype=float32)